## We need the optimal strategy to load/save models

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append(os.path.expandvars("/home/$USER/repos/fande/") )

In [3]:
import torch
from fande.ase import FandeCalc

predictor_loaded = torch.load("/data1/simulations/ML_models/predictor_forced.pt")
fande_calc = FandeCalc(predictor_loaded)

atoms = predictor_loaded.fdm.traj_train[0].copy()
atoms.calc = fande_calc

print( atoms.get_potential_energy(), atoms.get_forces() )


objects of fande.predict module imported...
objects of fande.data module imported...


/home/dlbox2/repos/fande/fande/predict/predictors_ase.py:571: UserWarning:

Calculating invariants...



Total number of frames is 1
Total number of batches is 1
Total length of traj is 1
Total number of batches 1
Calculating invariants on trajectory with librascal...


100%|██████████| 1/1 [00:00<00:00,  1.48it/s]
/home/dlbox2/repos/fande/fande/predict/predictors_ase.py:595: UserWarning:

Predicting...



0.0 [[-0.08429337 -0.02220154  0.04220963]
 [ 0.21712875  0.0256424   0.0921402 ]
 [-0.06847763  0.06993103 -0.05839157]
 ...
 [-0.0537529  -0.05883408 -0.06188393]
 [-0.10726929  0.14159775  0.09696579]
 [ 0.09921551 -0.02743912 -0.09516144]]


In [4]:
len(predictor_loaded.fdm.train_indices[5])

7024

In [6]:
predictor_loaded.fdm.soap_hypers

{'soap_type': 'PowerSpectrum',
 'interaction_cutoff': 3.0,
 'max_radial': 4,
 'max_angular': 4,
 'gaussian_sigma_constant': 0.3,
 'gaussian_sigma_type': 'Constant',
 'cutoff_function_type': 'RadialScaling',
 'cutoff_smooth_width': 0.1,
 'cutoff_function_parameters': {'rate': 1,
  'scale': 3.5,
  'exponent': 4,
  'interaction_cutoff': 3.0,
  'cutoff_smooth_width': 0.1},
 'radial_basis': 'GTO',
 'normalize': True,
 'compute_gradients': True,
 'expansion_by_species_method': 'structure wise'}

In [20]:
from ase import io

traj_295 = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_295_295K/md_trajectory.traj", index=":")
traj_355 = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_355_355K/md_trajectory.traj", index=":")

traj_295_2000K = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_295_2000K/md_trajectory.traj", index=":")
traj_355_2000K = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_355_2000K/md_trajectory.traj", index=":")
traj_295_2000K_forced = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_295_2000K_0075force/md_trajectory.traj", index=":")
traj_355_2000K_forced = io.read("/data1/simulations/datasets/rotors/high_temp_ML_training_data/results_triasine_ML_2000/struct_355_2000K_0075force/md_trajectory.traj", index=":")

In [13]:
from fande.utils.test import make_test_plots

In [21]:
save_dir_295 = "/data1/simulations/ML_models/test_plots/295"
save_dir_355 = "/data1/simulations/ML_models/test_plots/355"
save_dir_295_2000 = "/data1/simulations/ML_models/test_plots/295_2000"
save_dir_355_2000 = "/data1/simulations/ML_models/test_plots/295_2000"
save_dir_295_2000_forced = "/data1/simulations/ML_models/test_plots/295_2000_forced"
save_dir_355_2000_forced = "/data1/simulations/ML_models/test_plots/355_2000_forced"

In [27]:
trajectories = [traj_295, traj_355, traj_295_2000K, traj_355_2000K, traj_295_2000K_forced, traj_355_2000K_forced]
save_dirs = [save_dir_295, save_dir_355, save_dir_295_2000, save_dir_355_2000, save_dir_295_2000_forced, save_dir_355_2000_forced]

In [48]:
%%capture c

for traj, save_dir in zip(trajectories, save_dirs):
    make_test_plots(fande_calc, traj, save_dir=save_dir)